In [3]:
# Define Neo4j connections
from neo4j import GraphDatabase
host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'password'
driver = GraphDatabase.driver(host,auth=(user, password))

In [4]:
def drop_graph(name):
    with driver.session() as session:
        drop_graph_query = """
        CALL gds.graph.drop('{}');
        """.format(name)
        session.run(drop_graph_query)

In [5]:
# Import libraries
import pandas as pd

def read_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("0; 'Underlying socket connection gone (_ssl.c:1772)'")


### Graph schema
We will be using the Countries of the world dataset made available on Kaggle by Fernando Lasso. Looking at the acknowledgments, the data originates from the CIA's World Factbook. Unfortunately, the contributor did not provide the year the dataset was compiled. My guess is the year 2013, but I might be wrong. The dataset contains various metrics such as area size, population, infant mortality, and more about 227 countries of the world.

### Graph import
First, we need to download the dataset and copy it to the <code>$Neo4j/import</code> folder. For some reason, the numbers in the CSV file use a comma as a floating point instead of a dot (0,1 instead of 0.1). We need to preprocess the data to be able to cast the numbers to float in Neo4j. With the help of an APOC procedure <code>apoc.cypher.run</code>, we can preprocess and store the data in a single cypher query. <code>apoc.cypher.run</code> allows us to run independent subqueries within the main cypher query and is excellent for various use cases.

In [7]:
import_query = """

LOAD CSV WITH HEADERS FROM "file:///countries%20of%20the%20world.csv" as row
// cleanup the data and replace comma floating point with a dot
CALL apoc.cypher.run(
    "UNWIND keys($row) as key 
     WITH row,
          key,
          toFloat(replace(row[key],',','.')) as clean_value
          // exclude string properties
          WHERE NOT key in ['Country','Region'] 
          RETURN collect([key,clean_value]) as keys", 
     {row:row}) YIELD value
MERGE (c:Country{name:trim(row.Country)})
SET c+= apoc.map.fromPairs(value.keys)
MERGE (r:Region{name:trim(row.Region)})
MERGE (c)-[:PART_OF]->(r)

"""

read_query(import_query)

""


### Identify missing values
Another useful APOC procedure is <code>apoc.meta.nodeTypeProperties</code>. With it, we can examine the node property schema of the graph. We will use it to identify how many missing values each feature of the country has.

In [8]:
identify_missing_values_query = """

// Only look at properties of nodes labeled "Country"
CALL apoc.meta.nodeTypeProperties({labels:['Country']})
YIELD propertyName, propertyObservations, totalObservations
RETURN propertyName,
       (totalObservations - propertyObservations) as missing_value,
       (totalObservations - propertyObservations) / toFloat(totalObservations) as pct_missing_value
ORDER BY pct_missing_value DESC LIMIT 10

"""

read_query(identify_missing_values_query)

,propertyName,missing_value,pct_missing_value
0,Climate,22,0.096916
1,Literacy (%),18,0.079295
2,Industry,16,0.070485
3,Agriculture,15,0.066079
4,Service,15,0.066079
5,Deathrate,4,0.017621
6,Phones (per 1000),4,0.017621
7,Net migration,3,0.013216
8,Birthrate,3,0.013216
9,Infant mortality (per 1000 births),3,0.013216


It looks like we don't have many missing values. However, we will disregard features with more than four missing values from our further analysis for the sake of simplicity.
### High correlation filter
High correlation filter is a simple data dimensionality reduction technique. Features with high correlation are likely to carry similar information and are more linearly dependant. Using multiple features with related information can bring down the performance of various models and can be avoided by dropping one of the two correlating features.

In [9]:
high_correlation_query = """

// Only look at properties of nodes labeled "Country"
CALL apoc.meta.nodeTypeProperties({labels:['Country']})
YIELD propertyName, propertyObservations, totalObservations
WITH propertyName,
       (totalObservations - propertyObservations) as missing_value
// filter our features with more than 5 missing values
WHERE missing_value < 5 AND propertyName <> 'name'
WITH collect(propertyName) as features
MATCH (c:Country)
UNWIND features as feature
UNWIND features as compare_feature
WITH feature,
     compare_feature,
     collect(coalesce(c[feature],0)) as vector_1,
     collect(coalesce(c[compare_feature],0)) as vector_2
// avoid comparing with a feature with itself
WHERE feature < compare_feature
RETURN feature,
       compare_feature,
       gds.alpha.similarity.pearson(vector_1, vector_2) AS correlation
ORDER BY correlation DESC LIMIT 10

"""

read_query(high_correlation_query)

,feature,compare_feature,correlation
0,Birthrate,Infant mortality (per 1000 births),0.841210
1,GDP ($ per capita),Phones (per 1000),0.828151
2,Deathrate,Infant mortality (per 1000 births),0.661350
3,Area (sq. mi.),Population,0.469985
4,Birthrate,Deathrate,0.420948
5,GDP ($ per capita),Net migration,0.381256
6,Coastline (coast/area ratio),Crops (%),0.338594
7,Phones (per 1000),Pop. Density (per sq. mi.),0.280954
8,Coastline (coast/area ratio),Pop. Density (per sq. mi.),0.241690
9,Net migration,Phones (per 1000),0.236930


Interesting to see that birth rate and infant mortality are very correlated. The death rate is also quite correlated with infant mortality, so we will drop the birth and death rate but keep the infant mortality. The number of phones and net migration seems to be correlated with the GDP. We will drop them both as well and keep the GDP. We will also cut the population and retain both the area and population density, which carry similar information.
### Feature statistics
At this point, we are left with eight features. We will examine their distributions with the <code>apoc.agg.statistics</code> function. It calculates numeric statistics such as minimum, maximum, and percentile ranks for a collection of values.

In [10]:
feature_stats_query = """

// define excluded features
WITH ['name', 
      'Deathrate', 
      'Birthrate',
      'Phones (per 1000)',
      'Net migration', 
      'Population'] as excluded_features
CALL apoc.meta.nodeTypeProperties({labels:['Country']})
YIELD propertyName, propertyObservations, totalObservations
WITH propertyName,
       (totalObservations - propertyObservations) as missing_value
WHERE missing_value < 5 AND 
      NOT propertyName in excluded_features
// Reduce to a single row
WITH collect(propertyName) as potential_features
MATCH (c:Country)
UNWIND potential_features as potential_feature
WITH potential_feature, 
     apoc.agg.statistics(c[potential_feature],
                        [0.5,0.75,0.9,0.95,0.99]) as stats
RETURN potential_feature, 
       apoc.math.round(stats.min,2) as min, 
       apoc.math.round(stats.max,2) as max, 
       apoc.math.round(stats.mean,2) as mean, 
       apoc.math.round(stats.stdev,2) as stdev,
       apoc.math.round(stats.`0.5`,2) as p50,
       apoc.math.round(stats.`0.75`,2) as p75,
       apoc.math.round(stats.`0.95`,2) as p95,
       apoc.math.round(stats.`0.99`,2) as p99

"""

read_query(feature_stats_query)

,potential_feature,min,max,mean,stdev,p50,p75,p95,p99
0,Other (%),33.33,100.00,81.64,16.10,85.70,95.44,99.81,100.00
1,Arable (%),0.00,62.11,13.80,13.01,10.42,20.00,40.54,55.30
2,Crops (%),0.00,50.68,4.56,8.34,1.03,4.44,20.00,45.71
3,Coastline (coast/area ratio),0.00,870.66,21.17,72.13,0.73,10.32,92.31,310.69
4,Infant mortality (per 1000 births),2.29,191.19,35.51,35.31,20.97,55.51,103.32,143.64
5,Pop. Density (per sq. mi.),0.00,16271.50,379.05,1656.53,78.80,188.50,838.60,6482.22
6,GDP ($ per capita),500.00,55100.00,9689.85,10026.91,5500.03,15700.06,29600.12,37800.25
7,Area (sq. mi.),2.00,17075200.00,598227.59,1786336.93,86600.50,437074.00,2345424.00,9631424.00


The Federated state of Micronesia has the ratio of coast to area at 870, which is pretty impressive. On the other hand, there are a total of 44 countries in the world with zero coastlines. Another fun fact is that Greenland has a population density rounded to 0 per square mile with its 56361 inhabitants and 2166086 square miles. It might be a cool place to perform social distancing.
We can observe that most of the features appear to be descriptive, except for the Other (%), which is mostly between 80 and 100. Due to the low variance, we will ignore it in our further analysis.
### Populate the missing values
We are left with seven features that we are going to use to infer a similarity network between countries. One thing we need to do before that is to populate the missing values. We will use a simple method and fill in the missing values of the features with the average value of the region the country is part of.

In [11]:
populate_missing_values = """

UNWIND ["Arable (%)",
        "Crops (%)",
        "Infant mortality (per 1000 births)",
        "GDP ($ per capita)"] as feature
MATCH (c:Country)
WHERE c[feature] IS null
MATCH (c)-[:PART_OF]->(r:Region)<-[:PART_OF]-(other:Country)
WHERE other[feature] IS NOT null
WITH c,feature,avg(other[feature]) as avg_value
CALL apoc.create.setProperty(c, feature, avg_value) 
YIELD node
RETURN distinct 'missing values populated'

"""

read_query(populate_missing_values)

,'missing values populated'
0,missing values populated


### MinMax normalization
Last but not least, we have to normalize our features to prevent any single feature dominating over others due to a larger scale. We will use the simple MinMax method of normalization to rescale features between 0 and 1.

In [12]:
minmax_normalization = """

UNWIND ["Arable (%)",
        "Crops (%)",
        "Infant mortality (per 1000 births)",
        "GDP ($ per capita)",
        "Coastline (coast/area ratio)",
        "Pop. Density (per sq. mi.)",
        "Area (sq. mi.)"] as feature
MATCH (c:Country)
// Calculate the min and the max value for each feature
WITH max(c[feature]) as max,
     min(c[feature]) as min,
     feature
MATCH (c1:Country)
WITH c1,
    // define property name to store back results 
    "n_" + feature AS newKey,
    // normalize values
    (toFloat(c1[feature]) - min) / (max - min) as normalized_value
// store results to properties
CALL apoc.create.setProperty(c1, newKey, normalized_value) 
YIELD node
RETURN distinct 'normalization done'

"""

read_query(minmax_normalization)

,'normalization done'
0,normalization done


### Similarity network with cosine similarity
We have finished the data preprocessing and can focus on the data analysis part. The first step of the analysis is to infer a similarity network with the help of the cosine similarity algorithm. We build a vector for each country based on the selected features and compare the cosine similarity between each pair of countries. If the similarity is above the predefined threshold, we store back the results in the form of a relationship between the pair of similar nodes. Defining an optimal threshold is a mix of art and science, and you'll get better with practice. Ideally, you want to infer a sparse graph as community detection algorithms do not perform well on complete or dense graphs. In this example, we will use the similarityCutoff value of 0.8 (range between -1 and 1). Alongside the similarity threshold, we will also use the topK parameter to store only the top 10 similar neighbors. We do this to ensure a sparser graph.

In [13]:
cosine_similarity_query = """

MATCH (c:Country)
// build the vector from features
WITH id(c) as id, [c["n_Arable (%)"],
                   c["n_Crops (%)"],
                   c["n_Infant mortality (per 1000 births)"],
                   c["n_GDP ($ per capita)"],
                   c["n_Coastline (coast/area ratio)"],
                   c["n_Pop. Density (per sq. mi.)"],
                   c["n_Area (sq. mi.)"]] as weights
WITH {item:id, weights: weights} as countryData
WITH collect(countryData) as data
CALL gds.alpha.similarity.cosine.write({
    nodeProjection: '*',
    relationshipProjection: '*',
    similarityCutoff:0.8,
    topK:10,
    data: data})
YIELD nodes, similarityPairs
RETURN nodes, similarityPairs

"""

read_query(cosine_similarity_query)

,nodes,similarityPairs
0,227,2190


### Graph data science library
With Neo4j's Graph Data Science library, we can run more than 30 different graph algorithms directly in Neo4j. Algorithms are exposed as cypher procedures, similar to the APOC procedures we've seen above.
GDS uses a projection of the stored graph, that is entirely in-memory to achieve faster execution times. We can project a view of the stored graph utilizing the <code>gdn.graph.create</code> procedure. For more details on the GDS graph projection, check out my previous blog post. In this example, we will project nodes that have a label Country and relationships with a type SIMILAR.

In [15]:
create_graph = """CALL gds.graph.create('similarity_network','Country','SIMILAR');"""

read_query(create_graph)

,graphName,nodeProjection,relationshipProjection,nodeCount,relationshipCount,createMillis
0,similarity_network,"{'Country': {'label': 'Country', 'properties':...","{'SIMILAR': {'aggregation': 'DEFAULT', 'orient...",227,2190,6


### Weakly connected components
More often than not, we start the graph analysis with the weakly connected components algorithm. It is a community detection algorithm used to find disconnected networks or islands within our graph. As we are only interested in the count of disconnected components, we can run the stats variant of the algorithm.

In [16]:
wcc_query = """

CALL gds.wcc.stats('similarity_network')
YIELD componentCount, componentDistribution
RETURN componentCount, 
       componentDistribution.min as min,
       componentDistribution.max as max,
       componentDistribution.mean as mean,
       componentDistribution.p50 as p50,
       componentDistribution.p75 as p75,
       componentDistribution.p90 as p90

"""

read_query(wcc_query)

,componentCount,min,max,mean,p50,p75,p90
0,1,227,227,227.0,227,227,227


The algorithm found only a single component within our graph. This is a favorable outcome as disconnected islands can skew the results of various other graph algorithms.
### Louvain algorithm
Another community detection algorithm is the Louvain algorithm. In basic terms, densely connected nodes are more likely to form a community. It relies on the modularity optimization to extract communities. The modularity optimization is performed in two steps. The first step involves optimizing the modularity locally. In the second step, it aggregates nodes belonging to the same community into a single node and builds a new network from those aggregated nodes. These two steps are repeated iteratively until a maximum of modularity is attained. A subtle side effect of these iterations is that we can take a look at the community structure at the end of each iteration, hence the Louvain algorithm is regarded as a hierarchical community detection algorithm. To include hierarchical community results, we must set the <code>includeIntermediateCommunities</code> parameter value to true.

In [17]:
louvain_algo_query = """

CALL gds.louvain.write('similarity_network',  
    {maxIterations:20,
     includeIntermediateCommunities:true,
     writeProperty:'louvain'})
YIELD ranLevels, communityCount,modularity,modularities

"""

read_query(louvain_algo_query)

,ranLevels,communityCount,modularity,modularities
0,2,8,0.746745,"[0.726476512166135, 0.746744646692104]"


We can observe by the <code>ranLevels</code> value that the Louvain algorithm found two levels of communities in our network. On the final level, it found eight g. We can now examine the extracted communities of the last level and compare their feature averages.

In [18]:
final_level_communities ="""

MATCH (c:Country)
RETURN c.louvain[-1] as community,
       count(*) as community_size,
       avg(c['Arable (%)']) as pct_arable,
       avg(c['Crops (%)']) as pct_crops, 
       avg(c['Infant mortality (per 1000 births)']) as infant_mortality,
       avg(c['GDP ($ per capita)']) as gdp,
       avg(c['Coastline (coast/area ratio)']) as coastline,
       avg(c['Pop. Density (per sq. mi.)']) as population_density,
       avg(c['Area (sq. mi.)']) as area_size,
       collect(c['name'])[..3] as example_members
ORDER BY gdp DESC

"""

read_query(final_level_communities)

,community,community_size,pct_arable,pct_crops,infant_mortality,gdp,coastline,population_density,area_size,example_members
0,17,51,5.493529,1.366863,8.799216,21739.215686,45.333725,1127.390196,6.859381e+05,"[Guam, Hong Kong, Iceland]"
1,18,23,19.527101,3.297601,8.276522,19091.304348,11.173478,239.895652,3.163263e+05,"[Guadeloupe, Guernsey, Israel]"
2,38,40,31.581500,2.995000,20.513250,7385.000000,8.343250,195.035000,2.095605e+05,"[Hungary, India, Korea, North]"
3,61,20,4.255000,0.641000,32.653300,6123.000000,0.255000,26.485000,2.585060e+06,"[Guyana, Iran, Jordan]"
4,33,26,13.234231,22.198077,23.057591,4465.384615,66.923462,370.726923,3.770538e+04,"[Jamaica, Kiribati, Lebanon]"
5,27,10,22.116000,10.184000,55.267000,2100.000000,2.702000,180.400000,2.983568e+05,"[Guatemala, Haiti, Honduras]"
6,70,24,14.853575,1.090783,69.105833,1870.833333,3.340000,100.829167,3.194362e+05,"[Iraq, Kenya, Kyrgyzstan]"
7,31,33,4.050909,1.487273,93.983636,1445.454545,1.500000,25.312121,6.613687e+05,"[Guinea, Guinea-Bissau, Laos]"


Louvain algorithm found eight distinct communities within the similarity network. The biggest group has 51 countries as members and has the largest average GDP at almost 22 thousand dollars. They are second in infant mortality and the coastline ratio but lead in population density by a large margin. There are two communities with an average GDP of around 20 thousand dollars, and then we can observe a steep drop to 7000 dollars in third place. With the decline in GDP, we can also find the rise of infant mortality almost linearly. Another fascinating insight is that most of the more impoverished communities have little to no coastline.
### Find representatives of communities with PageRank
We can assess the top representatives of the final level communities with the PageRank algorithm. If we assume that each SIMILAR relationship is a vote of similarity between countries, the PageRank algorithm will assign the highest score to the most similar countries within the community. We will execute the PageRank algorithm for each community separately and consider only nodes and relationships within the given community. This can be easily achieved with cypher projection without any additional transformations.

In [19]:
top_representatives_query = """

WITH 'MATCH (c:Country) WHERE c.louvain[-1] = $community 
      RETURN id(c) as id' as nodeQuery,
     'MATCH (s:Country)-[:SIMILAR]->(t:Country) 
      RETURN id(s) as source, id(t) as target' as relQuery
MATCH (c:Country)
WITH distinct c.louvain[-1] as community, nodeQuery, relQuery
CALL gds.pageRank.stream({nodeQuery:nodeQuery,
                          relationshipQuery:relQuery, 
                          parameters:{community:community},
                          validateRelationships:False})
YIELD nodeId, score
WITH community, nodeId,score
ORDER BY score DESC
RETURN community, 
       collect(gds.util.asNode(nodeId).name)[..5] as top_5_representatives

"""

read_query(top_representatives_query)

,community,top_5_representatives
0,31,"[Mozambique, Central African Rep., Laos, Yemen..."
1,38,"[Macedonia, Mauritius, Hungary, Lithuania, Pol..."
2,17,"[Andorra, Cayman Islands, Sweden, Finland, Japan]"
3,18,"[Korea, South, Taiwan, United Kingdom, Netherl..."
4,33,"[Samoa, Tunisia, Cook Islands, Lebanon, Sri La..."
5,61,"[Egypt, Guyana, Peru, Namibia, Botswana]"
6,70,"[Azerbaijan, Benin, Kyrgyzstan, Burma, Senegal]"
7,27,"[Ghana, Uganda, Haiti, Burundi, Rwanda]"


### Hierarchical communities based on the Louvain algorithm
We mentioned before that the Louvain algorithm can be used to find hierarchical communities with the includeIntermediateCommunities parameter and that in our example, it found two levels of communities. We will now examine the groups of countries on the first level. A rule of thumb is that communities on a lower level will be more granular and smaller.

In [20]:
first_level_communities = """

MATCH (c:Country)
RETURN c.louvain[0] as community,
       count(*) as community_size,
       avg(c['Arable (%)']) as pct_arable,
       avg(c['Crops (%)']) as pct_crops, 
       avg(c['Infant mortality (per 1000 births)']) as infant_mortality,
       avg(c['GDP ($ per capita)']) as gdp,
       avg(c['Coastline (coast/area ratio)']) as coastline,
       avg(c['Pop. Density (per sq. mi.)']) as population_density,
       avg(c['Area (sq. mi.)']) as area_size,
       collect(c['name'])[..3] as example_members
ORDER BY gdp DESC

"""

read_query(first_level_communities)

,community,community_size,pct_arable,pct_crops,infant_mortality,gdp,coastline,population_density,area_size,example_members
0,17,22,10.367727,2.023182,6.101364,26154.545455,14.310455,185.300000,1.362027e+06,"[Guam, Ireland, Isle of Man]"
1,56,5,1.338000,0.202000,4.042000,23280.000000,123.566000,9856.420000,3.644000e+02,"[Hong Kong, Macau, Monaco]"
2,18,14,20.745952,1.208201,6.735714,21335.714286,9.730714,296.542857,2.993842e+05,"[Guernsey, Israel, Korea, South]"
3,11,20,1.182500,0.774500,12.119500,19465.000000,25.092000,141.675000,2.507723e+05,"[Iceland, Jersey, Kuwait]"
4,19,9,17.631111,6.547778,10.673333,15600.000000,13.417778,151.777778,3.426807e+05,"[Guadeloupe, Italy, Martinique]"
5,38,23,33.699130,2.114348,11.056522,10256.521739,13.452609,200.434783,9.362900e+04,"[Hungary, Latvia, Lithuania]"
6,75,4,5.435000,2.175000,12.982500,6900.000000,219.380000,326.175000,2.455000e+02,"[Nauru, N. Mariana Islands, Palau]"
7,61,20,4.255000,0.641000,32.653300,6123.000000,0.255000,26.485000,2.585060e+06,"[Guyana, Iran, Jordan]"
8,33,15,17.153333,14.770000,22.656912,4800.000000,27.435333,537.733333,4.220207e+04,"[Jamaica, Lebanon, Maldives]"
9,48,11,7.890000,32.327273,23.603971,4009.090909,120.770909,142.990909,3.157355e+04,"[Kiribati, Malaysia, Marshall Islands]"


As expected, there are almost twice as many communities on the first level compared to the second and final level. An exciting community formed in second place by the average GDP. It contains only five countries, which are quite tiny as their average area size is only 364 square miles. On the other hand, they have a very high population density of around 10000 people per square mile. Example members are Macau, Monaco, and Hong Kong.

In [21]:
drop_graph('similarity_network')